In [1]:
# CREDIT: Megan McCarthy
# COMPILED: Brandon Ma

## This program reads from a local directory and selects files that has log.lammps extension and
## checks if a certain column exists before generating the data after said column is confirmed to exist
## before converting it to .csv along with the file stem taken from the file's name without having to do it manually.

import os, sys
import numpy as np
import matplotlib.pyplot as plt

all_the_log_files = [f for f in os.listdir() if f.endswith('.lammps') and not os.path.isdir(f)]

store_numpy_arrays = []

for log_file in all_the_log_files:
    file_stem = log_file.replace('.log.lammps','')

    ## get the text from the log file
    with open(log_file, 'r') as f:
        txt = f.readlines()

    col_check_str = 'Step          Temp         v_strain      v_sigmaxx      v_sigmayy'
    end_check_str = 'Loop time of '
    columns = None
    start_idx, end_idx = 0, len(txt)

    for line_idx, line in enumerate(txt):
        if col_check_str in line:
            columns = line.strip().split()
            start_idx = line_idx + 1
        if end_check_str in line and columns:
            end_idx = line_idx - 1

    ## PRINT DEBUG
    # print(columns)
    # print(start_idx,end_idx)
    # print(data0[-1:])

    data0 = txt[start_idx:end_idx]
    data = [row for row in data0]
    columns_entry = None
        
    ## write the CSV file
    csv_name = f'{file_stem}.csv'
    with open(csv_name, 'w') as f:
        f.write(",".join(columns) + "\n")
        for start_idx in data:
            columns_entry = start_idx.strip().split()
            f.write(",".join(columns_entry) + "\n")
    
    ## Create an array to store each elements in a column
    arr = np.array([[float(element) for element in row.split()] for row in data0])
    store_numpy_arrays.append(arr)
    
    ssc = store_numpy_arrays[0] ## stress strain curve array.... Use counter???
    
    ## Takes the columns into the variables
    col_strain = ssc[:,2]
    col_sigmaxx = ssc[:,3] 
    col_sigmayy = ssc[:,4] 
    col_sigmazz = ssc[:,5] 
    
#     ## Print the columns entries for debugging purposes
#     print(col_strain)
#     print(col_sigmaxx)
#     print(col_sigmayy)
#     print(col_sigmazz)

## One row, three columns of graphs 
    
    for counter in store_numpy_arrays:
        fig, axes = plt.subplots(1, 3, figsize = [15,4])

        axes[0].plot(col_strain, col_sigmaxx,label="sigma_xx",c="k")
        axes[0].set_xlabel('strain')
        axes[0].set_ylabel('sigmaxx')

        axes[1].plot(col_strain, col_sigmayy,label="sigma_yy",c="r")
        axes[1].set_xlabel('strain')
        axes[1].set_ylabel('sigmayy')

        axes[2].plot(col_strain, col_sigmazz,label="sigma_zz",c="b")
        axes[2].set_xlabel('strain')
        axes[2].set_ylabel('sigmazz')
        
        ## Find the max stress fracture and append it to the title for references
        max_sigmazz = np.max(col_sigmazz)
        fig_info = f"Maximum value of sigmazz: {max_sigmazz} GPa"

        fig.suptitle(fig_info)

        ## Create a PNG of the graphs and save it
        png_name = f'{file_stem}_StressStrain.png'
        plt.savefig(png_name, transparent=True)

        ## Fit the graphs layout, then show and close. Required for normal scripts
        fig.tight_layout()
        plt.show()
        plt.close()     
    
    

NameError: name 'ssc' is not defined